# 라이브러리

In [1]:
import pandas as pd
import numpy as np
import re

# 확정된 함수들

In [2]:
# 패턴을 이용하여 저장 값을 리스트로 되돌리는 함수
def make_ls(c,i): # 인덱스 리스트와 순서를 받음
    matches = re.findall(r"'(.*?)'", c[i])
    return list(matches)

def clear_text(sentence):  # 목차 내용 정제 함수
    pattern = r'[\w가-힣]+'
    extracted = re.findall(pattern, sentence)
    result = ' '.join(extracted)
    return result
    
def delete_text(sentence):
    pattern = r'u3000|u000|u00|uf0ab|u2005'
    result = re.sub(pattern, '', sentence)
    return result
    
def split_pn(line): # 페이지 분리 함수
    a = re.sub(r'[^(X|IX|IV|V?I|Ⅱ|Ⅲ)]','', line)
    if line.split(' ')[-1].isnumeric() == True and a == '':
        page = line.split(' ')[-1]  
        line = ' '.join(line.split(' ')[:-1])
#         line = line.replace(line.split(' ')[-1]  ,'').strip() # 멀쩡한 애도 지워서
    else:
        page = np.NaN
    return line, page


# def classification_list(text, ex): # 판별 요소 설정 함수
#     pd_l,roma_l, ara_l = [],[],[]
#     for idx, line in enumerate(text):
#         a = re.sub(r'[^(X|IX|IV|V?I|Ⅱ|Ⅲ)]','', line)
#         # 숫자 판별
#         if line[0].isnumeric() == True: 
#             if a != '' and 'roma' not in pd_l: # 로마 숫자
#                 roma_l.append(idx) # 위치 저장
#                 pd_l.append('roma')
#             elif a != '' and 'roma' in pd_l: # 로마 숫자
#                 roma_l.append(idx) # 위치 저장
#             elif a == '' and 'ara' not in pd_l:
#                 ara_l.append(idx)
#                 pd_l.append('ara')
#             elif a == '' and 'ara' in pd_l:
#                 ara_l.append(idx)
#         elif line[0].isnumeric() != True:   # 포함되는 리스트
#             t = re.sub(r'[^a-zA-Z가-힣]','',line.lower())  # 기록 시에는 line을 기록해야 함
#             b = ex.match(t)
#             if b and b.group() not in pd_l:
#                 pd_l.append(b.group())
# #                 print(b.group(), text)
#             elif b and b.group() in pd_l:
#                 pass
#             else:
#                 pd_l.append('check')
#                 print(t)      
                
#     return pd_l, roma_l, ara_l

In [3]:
# 등록 함수 정의
def clear_idx(isbn, title, text, i, ex):
    code = isbn[i] # isbn
    title = title[i] # main_title
    if '목차 더보기' in text: # 잘못 들어간 단어 삭제(틀 전체로 접근하다 보니 같이 달려옴)
        text.remove('목차 더보기') 
    text = [clear_text(r) for r in text] # 이상한 문자 제거
    text = [delete_text(r) for r in text] # 특정 문자 제거
    pandan, roma, ara= classification_list2(text, ex) # 분류 판단 기준 생성 함수
    pandan = plus_pandan(pandan)
    print(pandan)
    if 'check' in pandan:
        isbn_ls.append(code)
        main_title.append(title)
        etc.append('check')
        for ls in [depth_1, depth_2, depth_3, depth_4, pages]:
            ls.append(np.NaN)
    elif 'check' not in pandan:
        if len(pandan) == 1:
            for num, line in enumerate(text):
                l,p = split_pn(line) # 페이지 번호랑 line 분류
                l = delete_text(l)
                depth_1.append(l)
                depth_2.append(np.NaN)
                depth_3.append(np.NaN)
                depth_4.append(np.NaN)
                etc.append(0) # 문제 없음
                pages.append(p)
                isbn_ls.append(code)
                main_title.append(title)
        elif len(pandan) == 2:
            for num, line in enumerate(text):
                a = re.sub(r'[^(Ⅹ|Ⅸ|Ⅳ|Ⅴ?Ⅰ|Ⅱ|Ⅲ)]','', line) 
                l,p = split_pn(line)
                if l[0].isnumeric() == True and l[1] in ['부', '절','편','강']: # 숫자분류
                    check = 'num'
                elif l[0].isnumeric() == True  and l[1] not in ['부', '절','편','강'] and a == '': # 아라비아
                    check = 'ara'
                elif l[0].isnumeric() == True and a != '': # roma
                    check = 'roma'
                elif l[0].isnumeric() != True:
                    b = re.sub(r'[^a-zA-Z가-힣]','',line.lower())
                    b = ex.match(b)
                    if b:
                        check = b.group()
#                         store_new(pandan, check, l, p, code, title) # store 수정중
                    else:
                        print(num,'적용 X')
                store_new(pandan, check, l, p, code, title) # store 수정중
        elif len(pandan) == 3:
            for num, line in enumerate(text):
                a = re.sub(r'[^(Ⅹ|Ⅸ|Ⅳ|Ⅴ?Ⅰ|Ⅱ|Ⅲ)]','', line)
                l,p = split_pn(line)
                if l[0].isnumeric() == True and l[1] in ['부', '절','편','강']: # 숫자분류
                    check = 'num'
                elif l[0].isnumeric() == True  and l[1] not in ['부', '절','편','강'] and a == '': # 아라비
                    check = 'ara'
                elif l[0].isnumeric() == True and a != '': # roma
                    check = 'roma'
                elif l[0].isnumeric() != True:
                    b = re.sub(r'[^a-zA-Z가-힣]','',line.lower())
                    b = ex.match(b)
                    
                    if b:
                        print(num, b.group())
                        check = b.group()
#                         store_new(pandan, check, l, p, code, title) # store 수정중
                        print(pandan)
                    else:
                        print(num,'적용 X')
                store_new(pandan, check, l, p, code, title) # store 수정중
                        
        elif len(pandan) >= 4:
            for num, line in enumerate(text):
                a = re.sub(r'[^(Ⅹ|Ⅸ|Ⅳ|Ⅴ?Ⅰ|Ⅱ|Ⅲ)]','', line)
                l,p = split_pn(line)
                if l[0].isnumeric() == True and l[1] in ['부', '절','편','강']: # 숫자분류
                    check = 'num'
                elif l[0].isnumeric() == True  and l[1] not in ['부', '절','편','강'] and a == '': # 아라비
                    check = 'ara'
                elif l[0].isnumeric() == True and a != '': # roma
                    check = 'roma'
                elif l[0].isnumeric() != True:
                    b = re.sub(r'[^a-zA-Z가-힣]','',line.lower())
                    b = ex.match(b)
                    if b:
                        check = b.group()
#                         store_new(pandan, check, l, p, code, title) # store 수정중
                    else:
                        print(num,'적용 X')
                store_new(pandan, check, l, p, code, title) # store 수정중

# 수정중인 함수

In [4]:
def classification_list2(text, ex): # 판별 요소 설정 함수
    pd_l,roma_l, ara_l = [],[],[]
    for idx, line in enumerate(text):
        a = re.sub(r'[^(X|IX|IV|V?I|Ⅱ|Ⅲ)]','', line)
        # 숫자 판별
        if line[0].isnumeric() == True: 
            if a != '' and 'roma' not in pd_l: # 로마 숫자
                roma_l.append(idx) # 위치 저장
                pd_l.append('roma')
            elif a != '' and 'roma' in pd_l: # 로마 숫자
                roma_l.append(idx) # 위치 저장
            elif a == '' and line[1] in ['부', '절','편','강'] and 'num' not in pd_l :
                ara_l.append(idx)
                pd_l.append('num') # 분류랑 숫자 결합
            elif a == '' and line[1] in ['부', '절','편','강'] and 'num' in pd_l:
#                 ara_l.append(idx)
                print('num')
            elif a == '' and line[1] not in ['부', '절','편','강'] and 'ara' not in pd_l :
                ara_l.append(idx)
                pd_l.append('ara') # 분류랑 숫자 결합
            elif a == '' and line[1] not in ['부', '절','편','강'] and 'ara' in pd_l:
                ara_l.append(idx)
        elif line[0].isnumeric() != True:   # 포함되는 리스트
            t = re.sub(r'[^a-zA-Z가-힣]','',line.lower())  # 기록 시에는 line을 기록해야 함
            b = ex.match(t)
            if b and b.group() not in pd_l:
                pd_l.append(b.group())
#                 print(b.group(), text)
            elif b and b.group() in pd_l:
                pass
            else:
                pd_l.append('check')
                print(t)      
                
    return pd_l, roma_l, ara_l

In [5]:
# 연습
ls = [1,2,['총론', '각론'],'c','각론']
ls.remove('각론')
ls

[1, 2, ['총론', '각론'], 'c']

In [6]:
# 총론 각론 인식 해결
def plus_pandan(pd_l):
    if '총' in pd_l and '각' in pd_l:
        i = pd_l.index('총')
        pd_l[i] = ['총', '각']
        pd_l.remove('각')
    elif '총론' in pd_l and '각론' in pd_l:
        i = pd_l.index('총론')
        pd_l[i] = ['총론', '각론']
        pd_l.remove('각론')
#     elif '총론' in pd_l and '각론' in pd_l and '부록' in pd_l:
#         i = pd_l.index('총론')
#         pd_l[i] = ['총론', '각론', '부록']
    elif '형법총론' in pd_l and '형법각론' in pd_l:
        i = pd_l.index('형법총론')
        pd_l[i] = ['형법총론', '형법각론']
        pd_l.remove('형법각론')
    elif '헌법총론' in pd_l and '헌법각론' in pd_l:
        i = pd_l.index('헌법총론')
        pd_l[i] = ['헌법총론', '헌법각론']
        pd_l.remove('헌법각론')
    else:
        pass
    return pd_l

In [7]:
def store_new(pandan, check, l, p, code, title):    
    l = delete_text(l)
    if type(pandan[0]) == list:
        try:
            print(pandan[0].index(check))
            depth_dict[f'{0}'].append(l)
            for ls in [1,2,3]:
                depth_dict[f'{ls}'].append(np.NaN)
            etc.append(0)
            pages.append(p)
            isbn_ls.append(code)
            main_title.append(title)
        except:
            ii = pandan.index(check)
            if ii >= 3:
                ii = 3
            else:
                ii = ii
            depth_dict[f'{ii}'].append(l)
            check_ls = [0,1,2,3]
            check_ls.remove(ii)
            for ls in check_ls:
                
                depth_dict[f'{ls}'].append(np.NaN)
            etc.append(0)
            pages.append(p)
            isbn_ls.append(code)
            main_title.append(title)
    elif type(pandan[1]) == list:  # 이부분 추가됨
        try:
            print(pandan[1].index(check))
            depth_dict[f'{1}'].append(l)
            for ls in [0,2,3]:
                depth_dict[f'{ls}'].append(np.NaN)
            etc.append(0)
            pages.append(p)
            isbn_ls.append(code)
            main_title.append(title)
        except:
            ii = pandan.index(check)
            if ii >= 3:
                ii = 3
            else:
                ii = ii
            depth_dict[f'{ii}'].append(l)
            check_ls = [0,1,2,3]
            check_ls.remove(ii)
            for ls in check_ls:
                depth_dict[f'{ls}'].append(np.NaN)
            etc.append(0)
            pages.append(p)
            isbn_ls.append(code)
            main_title.append(title)
    else:
        ii = pandan.index(check)
        if ii >= 3:
            ii = 3
        else:
            ii = ii
        depth_dict[f'{ii}'].append(l)
        check_ls = [0,1,2,3]
        check_ls.remove(ii)
        for ls in check_ls:
            depth_dict[f'{ls}'].append(np.NaN)
        etc.append(0)
        pages.append(p)
        isbn_ls.append(code)
        main_title.append(title)

In [ ]:
# def store_new(pandan, check, l, p, code, title):    
#     l = delete_text(l)
#     if type(pandan[0]) == list:
#         try:
#             print(pandan[0].index(check))
#             depth_dict[f'{0}'].append(l)
#             for ls in [1,2,3]:
#                 depth_dict[f'{ls}'].append(np.NaN)
#             etc.append(0)
#             pages.append(p)
#             isbn_ls.append(code)
#             main_title.append(title)
#         except:
#             ii = pandan.index(check)
#             if ii >= 3:
#                 ii = 3
#             else:
#                 ii = ii
#             depth_dict[f'{ii}'].append(l)
#             check_ls = [0,1,2,3]
#             check_ls.remove(ii)
#             for ls in check_ls:
#                 depth_dict[f'{ls}'].append(np.NaN)
#             etc.append(0)
#             pages.append(p)
#             isbn_ls.append(code)
#             main_title.append(title)
#     else:
#         ii = pandan.index(check)
#         if ii >= 3:
#             ii = 3
#         else:
#             ii = ii
#         depth_dict[f'{ii}'].append(l)
#         check_ls = [0,1,2,3]
#         check_ls.remove(ii)
#         for ls in check_ls:
#             depth_dict[f'{ls}'].append(np.NaN)
#         etc.append(0)
#         pages.append(p)
#         isbn_ls.append(code)
#         main_title.append(title)

### 시행착오

In [104]:
tet = make_ls(text, 3)

In [38]:
t = isbn.to_list()
t

[9791166184604,
 9791197865367,
 9791169873437,
 9791192989099,
 9791192989082,
 9788952645432,
 9791166871238,
 9788952645425,
 9791163485827,
 9791165234355,
 9791163485810,
 9791167432629,
 9791165234331,
 9791198342201,
 9791165234348,
 9791165234317,
 9791160492699,
 9791162933916,
 9791192930183,
 9791130317625,
 9788952645289,
 9791169992442,
 9791169992459,
 9791169992466,
 9791169660488,
 9791169992695,
 9791193144008,
 9791166871214,
 9791167432612,
 9791189599850,
 9791198221889,
 9788952645241,
 9788952645241,
 9788958192824,
 9788958192855,
 9788958192848,
 9788958192862,
 9788958192831,
 9791160492651,
 9791192070803,
 9791163485780,
 9791138353106,
 9791138353106,
 9791163131441,
 9791163131434,
 9791192152516,
 9791165234270,
 9791191870725,
 9791169660600,
 9788952645173,
 9788952645173,
 9791163485766,
 9791165234188,
 9791165234195,
 9791166871207,
 9791191612127,
 9791191817317,
 9791191817300,
 9791168813373,
 9791165234232,
 9791163485759,
 9791167733191,
 9791165

In [223]:
text[0]

"['/제1편/ 헌법총론', '제1장 헌법과 헌법학 2', '제1절 헌법의 의의 2', '제1항 헌법의 개념과 분류 3', '제2항 헌법의 특성 6', '제2절 헌법학 연구 8', '제1항 헌법의 해석 8', '제2항 헌법관 12', '제3절 헌법의 제정 12', '제4절 헌법의 개정 13', '제5절 헌법의 변천 20', '제6절 헌법의 수호 20', '제2장 대한민국헌법총설 24', '제1절 대한민국헌정사 24', '제2절 대한민국의 국가형태 32', '제3절 대한민국의 구성요소 33', '제4절 한국헌법의 기본원리 46', '제1항 한국헌법의 전문 46', '제2항 국민주권의 원리 51', '제3항 민주주의의 원리 53', '제4항 법치주의의 원리 55', '제5항 사회국가의 원리 70', '제6항 문화국가의 원리 77', '제7항 평화국가의 원리 79', '제5절 정당제도 83', '제6절 선거제도 92', '제7절 공무원제도 102', '제8절 지방자치제도 105', '/제2편/ 기본권론', '제1장 기본권 총론 112', '제1절 기본권보장의 역사 112', '제2절 기본권의 성격 113', '제3절 기본권의 주체 115', '제4절 기본권의 포기 124', '제5절 기본권의 효력 124', '제6절 기본권의 보호의무 126', '제7절 기본권의 갈등 131', '제8절 기본권의 제한과 그 한계 136', '제9절 기본권의 보호 149', '제2장 인간의 존엄과 가치ㆍ행복추구권ㆍ평등권 151', '제1절 인간의 존엄과 가치ㆍ행복추구권 151', '제2절 평등권 164', '제3장 자유권적 기본권 176', '제1절 인신에 관한 자유 176', '제1항 생명권 176', '제2항 신체를 훼손당하지 않을 권리 178', '제3항 신체의 자유 179', '제2절 사생활영역의 자유 217', '제1항 사생활의 비밀과 자유 217', '제2항 주거의 자유 228', '제3항 거주ㆍ이전의 자유 229', '제4항 통신의 자유 231', '제3절 정신생활영역의 

In [221]:
t[21]

9791169992442

In [217]:
t.index(9791169992442
)


21

In [218]:
tet = make_ls(text, 21)
if '목차 더보기' in tet: # 잘못 들어간 단어 삭제(틀 전체로 접근하다 보니 같이 달려옴)
    tet.remove('목차 더보기') 
tet = [clear_text(r) for r in tet] # 이상한 문자 제거
tet = [delete_text(r) for r in tet]
pandan, roma, ara= classification_list2(tet, ex) # 분류 판단 기준 생성 함수
pandan = plus_pandan(pandan)
print(pandan)

['ara', '제장', '제절']


In [220]:
# 필요한 전처리
# title = title[58] # main_title
tet = make_ls(text, 21)

if '목차 더보기' in tet: # 잘못 들어간 단어 삭제(틀 전체로 접근하다 보니 같이 달려옴)
    tet.remove('목차 더보기') 
tet = [clear_text(r) for r in tet] # 이상한 문자 제거
tet = [delete_text(r) for r in tet]
for i in range(len(tet)):
    print('-', tet[i])
    l,p = split_pn(tet[i])
    print(l)
# ㅣ= 
# ㅣ= delete_text(l)

- 1권 총론
1권 총론
- 제1장 행정의 의의
제1장 행정의 의의
- 제 1 절 행정의 개념
제 1 절 행정의 개념
- 제 2 절 통치행위
제 2 절 통치행위
- 제2장 경찰의 의의
제2장 경찰의 의의
- 제 1 절 경찰의 개념과 특징
제 1 절 경찰의 개념과 특징
- 제 2 절 형식적 의미의 경찰과 실질적 의미의 경찰
제 2 절 형식적 의미의 경찰과 실질적 의미의 경찰
- 제 3 절 경찰의 구분
제 3 절 경찰의 구분
- 제 4 절 경찰의 임무 및 수단
제 4 절 경찰의 임무 및 수단
- 제 5 절 경찰권
제 5 절 경찰권
- 제 6 절 경찰의 관할
제 6 절 경찰의 관할
- 제 7 절 경찰활동의 기본이념
제 7 절 경찰활동의 기본이념
- 제3장 한국경찰사
제3장 한국경찰사
- 제 1 절 서설
제 1 절 서설
- 제 2 절 갑오개혁 이전의 경찰제도
제 2 절 갑오개혁 이전의 경찰제도
- 제 3 절 갑오개혁 이후의 경찰제도
제 3 절 갑오개혁 이후의 경찰제도
- 제 4 절 일제강점기의 경찰제도
제 4 절 일제강점기의 경찰제도
- 제 5 절 미군정기의 경찰제도
제 5 절 미군정기의 경찰제도
- 제 6 절 대한민국 정부수립 이후의 경찰제도
제 6 절 대한민국 정부수립 이후의 경찰제도
- 제 7 절 한국경찰사 관련 주요 인물
제 7 절 한국경찰사 관련 주요 인물
- 제4장 경찰법학
제4장 경찰법학
- 제 1 절 경찰법학 일반
제 1 절 경찰법학 일반
- 제 2 절 경찰조직법
제 2 절 경찰조직법
- 제 3 절 경찰공무원법
제 3 절 경찰공무원법
- 제 4 절 경찰작용법
제 4 절 경찰작용법
- 제5장 경찰구제법
제5장 경찰구제법
- 제 1 절 경찰구제의 의의
제 1 절 경찰구제의 의의
- 제 2 절 경찰상 손해배상
제 2 절 경찰상 손해배상
- 제 3 절 경찰상 손실보상
제 3 절 경찰상 손실보상
- 제 4 절 행정심판법
제 4 절 행정심판법
- 제 5 절 행정소송법
제 5 절 행정소송법
- 제6장 경찰관리
제6장 경찰관리
- 제 1 절 경찰관리 일반
제 1 절 

# 임시로 뽑아본거 (new_index_test)

In [140]:
text[51]

"['1부 암수위(암기·숫자·위원회)', 'PART 01 총론', 'CHAPTER 01 경찰학의 기초이론 6', '1 경찰개념의 변천 ·················································································································6', '2 법치국가 경찰개념을 포함한 법령/판례(시대순) ································································6', '3 경찰의 분류 ························································································································6', '4 실정법상 경찰 임무 ·············································································································6', '5 사회공공의 안녕과 질서에 대한 위험방지(경찰의 기본적 임무) ·········································6', '6 경찰헌장 ·····························································································································7', '7 경찰의 일탈과 부패 ··············································································································7', '8 적극행정의 정의와 적극행정위원회 ··················································································

In [14]:
depth_dict = {'0':depth_1,'1':depth_2,'2':depth_3,'3':depth_4}

# 실행
for i in range(len(text)):
    try:
        tet = make_ls(text, i)
        clear_idx(isbn, title, tet, i, ex)
#         clear_idx2(a,b,text,i)
        print(i)
    except:
        print('i')
        isbn_ls.append(isbn[i])
        main_title.append(title[i])
        etc.append(2)  # 데이터가 존재하지 않는 경우
        for k in [depth_1, depth_2,depth_3,depth_4,pages]:
            k.append(np.NaN)

['제편', '제장', '제절', '제항']
0
['제편', '제장', '제절']
0 제편
['제편', '제장', '제절']
1 제장
['제편', '제장', '제절']
2 제절
['제편', '제장', '제절']
3 제절
['제편', '제장', '제절']
4 제장
['제편', '제장', '제절']
5 제절
['제편', '제장', '제절']
6 제절
['제편', '제장', '제절']
7 제절
['제편', '제장', '제절']
8 제절
['제편', '제장', '제절']
9 제편
['제편', '제장', '제절']
10 제장
['제편', '제장', '제절']
11 제절
['제편', '제장', '제절']
12 제절
['제편', '제장', '제절']
13 제절
['제편', '제장', '제절']
14 제장
['제편', '제장', '제절']
15 제절
['제편', '제장', '제절']
16 제절
['제편', '제장', '제절']
17 제장
['제편', '제장', '제절']
18 제절
['제편', '제장', '제절']
19 제절
['제편', '제장', '제절']
1
빠른정답찾기
['전범위', '제회', '기출', 'ara', '정답', 'check', '부']
2
['최근', ['총론', '각론'], '제강', '목']
0
1
3
['목', '제강']
4
['part', 'chapter']
5
['ara']
6
['chapter', '제절']
7
['제편', '제장', '제절']
0 제편
['제편', '제장', '제절']
1 제장
['제편', '제장', '제절']
2 제절
['제편', '제장', '제절']
3 제절
['제편', '제장', '제절']
4 제절
['제편', '제장', '제절']
5 제장
['제편', '제장', '제절']
6 제절
['제편', '제장', '제절']
7 제절
['제편', '제장', '제절']
8 제절
['제편', '제장', '제절']
9 제절
['제편', '제장', '제절']
10 제장
['제편', '제장', '제절']
11 제절
['제편', '제장',

In [15]:

df_ls = pd.DataFrame({'isbn':isbn_ls, 'main_title':main_title,'depth_1':depth_1, 'depth_2':depth_2,\
                      'depth_3':depth_3, 'depth_4': depth_4,'pages':pages,'etc':etc})
df_ls.to_excel('C:/police_data/06_27_test3(23day).xlsx',encoding = 'utf-8-sig', index=False) 

C:\Data\conda\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [32]:

df_ls = pd.DataFrame({'isbn':isbn_ls, 'main_title':main_title,'depth_1':depth_1, 'depth_2':depth_2,\
                      'depth_3':depth_3, 'depth_4': depth_4,'pages':pages,'etc':etc})
df_ls.to_excel('C:/police_data/new_index_test.xlsx',encoding = 'utf-8-sig', index=False) 

C:\Data\conda\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [117]:
# 분류 키워드 추가한 버전
df_ls = pd.DataFrame({'isbn':isbn_ls, 'main_title':main_title,'depth_1':depth_1, 'depth_2':depth_2,\
                      'depth_3':depth_3, 'depth_4': depth_4,'pages':pages,'etc':etc})
df_ls.to_excel('C:/police_data/new_index_test2.xlsx',encoding = 'utf-8-sig', index=False) 

C:\Data\conda\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [176]:
# 숫자+분류코드 인 형태를 수정함(classification 2)
df_ls = pd.DataFrame({'isbn':isbn_ls, 'main_title':main_title,'depth_1':depth_1, 'depth_2':depth_2,\
                      'depth_3':depth_3, 'depth_4': depth_4,'pages':pages,'etc':etc})
df_ls.to_excel('C:/police_data/new_index_test3.xlsx',encoding = 'utf-8-sig', index=False) 

C:\Data\conda\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [208]:
# 위상태에서 키워드를 더 추가한 버전 + 총론 리스트가 두번쨰 위치에 있는경우 추가함
df_ls = pd.DataFrame({'isbn':isbn_ls, 'main_title':main_title,'depth_1':depth_1, 'depth_2':depth_2,\
                      'depth_3':depth_3, 'depth_4': depth_4,'pages':pages,'etc':etc})
df_ls.to_excel('C:/police_data/new_index_test4.xlsx',encoding = 'utf-8-sig', index=False) 

C:\Data\conda\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [216]:
# 총론 리스트가 두번쨰 위치에 있는경우 추가함
df_ls = pd.DataFrame({'isbn':isbn_ls, 'main_title':main_title,'depth_1':depth_1, 'depth_2':depth_2,\
                      'depth_3':depth_3, 'depth_4': depth_4,'pages':pages,'etc':etc})
df_ls.to_excel('C:/police_data/new_index_test5.xlsx',encoding = 'utf-8-sig', index=False) 

C:\Data\conda\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


# 필요 변수 & 데이터

In [13]:
data = pd.read_excel("C:\police_data\police_book_idx_type.xlsx")

# 변수 설정
isbn, title, text = data['isbn'], data['교재명'], data['index']


isbn_ls, main_title, depth_1, depth_2, depth_3, depth_4, etc, pages = [],[],[],[],[],[],[], []
depth_dict = {'0':depth_1,'1':depth_2,'2':depth_3,'3':depth_4}

In [8]:
data = pd.read_excel("C:\경찰교재 수집 데이터\police_idx_data.xlsx")

# 변수 설정
isbn, title, text = data['isbn'], data['main_title'], data['index']


isbn_ls, main_title, depth_1, depth_2, depth_3, depth_4, etc, pages = [],[],[],[],[],[],[], []
depth_dict = {'0':depth_1,'1':depth_2,'2':depth_3,'3':depth_4}

In [30]:
# 1차
ex = re.compile("제강|제항|제장|제회|강|목|장|항|회|사례|총론|형법총론|헌법총론|각론|제절|제편|형법각론|헌법각론|part|chapter|day|section|theme")

In [145]:
# 2차
ex = re.compile("제절|제편|제강|제항|제장|제회|제부|강|목|장|항|회|부|사례|총론|형법총론|헌법총론|각론|형법각론|헌법각론|부록|키워드|최근|실전|모의|문제|정답|part|chapter|day|section|theme|case|keyword")

In [9]:
# 3차
ex = re.compile("제절|제편|제강|제항|제장|제회|제부|제과|제관|강|목|장|항|회|부|사례|총론|형법총론|헌법총론|각론|형법각론|헌법각론|부록|키워드|최근|실전|모의|문제|정답|해설|차례|전범위|기출|참고|판례|핵심|part|chapter|day|section|theme|case|keyword|point|q|question|contents")

In [224]:
"제절|제편|제강|제항|제장|제회|제부|제과|제관|강|목|장|항|회|부|사례|총론|형법총론|헌법총론|각론|
형법각론|헌법각론|부록|키워드|최근|실전|모의|문제|정답|해설|차례|전범위|기출|참고|판례|핵심|
part|chapter|day|section|theme|case|keyword|point|q|question|contents"

SyntaxError: unterminated string literal (detected at line 1) (1896636928.py, line 1)

In [225]:
a = "제절|제편|제강|제항|제장|제회|제부|제과|제관|강|목|장|항|회|부|사례|총론|형법총론|헌법총론|각론|형법각론|헌법각론|부록|키워드|최근|실전|모의|문제|정답|해설|차례|전범위|기출|참고|판례|핵심|part|chapter|day|section|theme|case|keyword|point|q|question|contents"
a.replace('|',', ')

'제절, 제편, 제강, 제항, 제장, 제회, 제부, 제과, 제관, 강, 목, 장, 항, 회, 부, 사례, 총론, 형법총론, 헌법총론, 각론, 형법각론, 헌법각론, 부록, 키워드, 최근, 실전, 모의, 문제, 정답, 해설, 차례, 전범위, 기출, 참고, 판례, 핵심, part, chapter, day, section, theme, case, keyword, point, q, question, contents'